In [1]:
# Access parent directories
from sys import path
from os.path import abspath
path += [abspath("../"*i) for i in (1, 2, 3)]

In [2]:
from QLab import SimpleAgent, QNetwork
from pacman import RandomAgent, ChaserAgent
import torch
from torch import nn
import torch.nn.functional as F

def flatten_state(state):
	if state is None:
		return None
	dist = state[3]
	pacman = torch.zeros_like(dist)
	pacman[dist == 0] = 1000
	state = state[0] - state[1] + 10*state[2] + pacman
	return state

class DeepNetwork(QNetwork):
	def __init__(self, h, w):
		super().__init__()
		self.connected1 = nn.Linear(h * w, 128)
		self.connected2 = nn.Linear(128, 256)
		self.connected3 = nn.Linear(256, 128)
		self.connected4 = nn.Linear(128, 4)

	def forward(self, x):
		x = torch.flatten(x, 1)
		x = F.relu(self.connected1(x))
		x = F.relu(self.connected2(x))
		x = F.relu(self.connected3(x))
		x = F.relu(self.connected4(x))
		return x

class RescaledRewardsAgent(SimpleAgent):
	def __call__(self, state):
		super().__call__(flatten_state(state))
		
	def feedback(self, state, action, reward, new_state):
		state = flatten_state(state)
		new_state = flatten_state(new_state)
		self.memory.push(state, action, reward, new_state)
		self.optimizer(gamma = 0.9)

height, width = (9,9)
shnet = DeepNetwork(height, width)
agent = RescaledRewardsAgent(shnet)

pygame 2.1.2 (SDL 2.0.16, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# agent.load()

In [4]:
agent.train(
	episodes=10000,
	update_frq=100,
	plot=True,
	live=False,
	ghosts=[RandomAgent(), ChaserAgent()],
	map = "tiny.map")
agent.save()

  0%|          | 0/10000 [00:00<?, ?it/s]


TypeError: 'NoneType' object is not subscriptable

In [ ]:
agent.epsilon = 0
agent.train(live=True, episodes=10, ghosts=[RandomAgent(), ChaserAgent()], map = "tiny.map")

  0%|          | 0/10 [00:00<?, ?it/s]